In [ ]:
"""
Merge genes.
"""
import os
from data_preparation import merge_csv
for c in range(4, 24):
    chr = f"chr{c+1}"
    src_dir = os.path.join("workspace", "seq2seq-stride.384", chr)
    dest_file = os.path.join("workspace", "seq2seq-stride.384", chr, "bundle.csv")
    src_files = [os.path.join(src_dir, fname) for fname in os.listdir(src_dir)] 
    status = merge_csv(src_files, dest_file)
    if not status:
        print(f"Something wrong with merging files in directory {src_dir}.")
        raise Exception("something wrong.")


In [ ]:
"""
Make sample from bundle.csv.
"""
from utils.utils import create_n_sample
import os

n_sample = 100
for c in range(24):
    bundle_csv = os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}", "bundle.csv")
    bundle_sample_csv = os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}", "bundle.sample.csv")
    create_n_sample(bundle_csv, n_sample, bundle_sample_csv)

In [ ]:
"""
Make train, validation, and test bundle for each chr.
"""
from utils.utils import split_and_store_csv
import os

fractions = [0.7, 0.2, 0.1]
file_types = ["train", "validation", "test"]
dest_dir = [os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}") for c in range(24)]
src_files = [os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}", "bundle.csv") for c in range(24)]
dest_train_files = [os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}", "bundle.train.csv") for c in range(24)]
dest_valid_files = [os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}", "bundle.validation.csv") for c in range(24)]
dest_test_files = [os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}", "bundle.test.csv") for c in range(24)]

for src, dest_train, dest_valid, dest_test in zip(src_files, dest_train_files, dest_valid_files, dest_test_files):
    split_and_store_csv(src, fractions, [
        dest_train, dest_valid, dest_test
    ])

In [ ]:
"""
Merge bundle.sample.csv from every chr.
"""
from data_preparation import merge_csv
import os
chr_bundle_sample_csvs = [os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}", "bundle.sample.csv") for c in range(24)]
merged_bundle_sample = os.path.join("workspace", "seq2seq-stride.384", "bundle.sample.csv")
merge_csv(chr_bundle_sample_csvs, merged_bundle_sample)

In [ ]:
"""
Merge bundle from every chr.
"""
from data_preparation import merge_csv
import os
chr_bundle_csvs = [os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}", "bundle.csv") for c in range(24)]
merged_bundle = os.path.join("workspace", "seq2seq-stride.384", "bundle.csv")
merge_csv(chr_bundle_csvs, merged_bundle)

In [ ]:
"""
Merge bundle train, validation, and test from every chr.
"""
from data_preparation import merge_csv
import os

for t in ["train", "validation", "test"]:
    chr_bundle_csvs = [os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}", f"bundle.{t}.csv") for c in range(24)]
    merged_bundle = os.path.join("workspace", "seq2seq-stride.384", f"bundle.{t}.csv")
    merge_csv(chr_bundle_csvs, merged_bundle)

In [ ]:
"""
Merge bundle training, validation, and test from every chr.
"""
from data_preparation import merge_csv
import os
for t in ["train", "validation", "test"]:
    chr_bundle_csvs = [os.path.join("workspace", "seq2seq-stride.384", f"chr{c+1}", f"bundle.{t}.csv") for c in range(24)]
    merged_bundle = os.path.join("workspace", "seq2seq-stride.384", f"bundle.{t}.csv")
    merge_csv(chr_bundle_csvs, merged_bundle)

In [ ]:
from data_preparation import kmer
import pandas as pd
import os

_cols = ['sequence', 'label']
_stride = 256
_length = 512
for c in ['chr{}'.format(i+1) for i in range(24)]:
    genes_dir_by_chr = os.path.join('data', 'genome', 'sequential-labelling-positive-strand', c)
    genes_expansion_dir_by_chr = os.path.join('workspace', f"sequential-labelling-stride.{_stride}", c)
    if not os.path.exists(genes_expansion_dir_by_chr):
        os.makedirs(genes_expansion_dir_by_chr, exist_ok=True)
    for gene_file in os.listdir(genes_dir_by_chr):
        gene_file_path = os.path.join(genes_dir_by_chr, gene_file)
        gene_expansion_file_path = os.path.join(genes_expansion_dir_by_chr, f"{gene_file.split('.')[0]}.expanded.csv")
        if os.path.exists(gene_expansion_file_path):
            os.remove(gene_expansion_file_path)
        target_file = open(gene_expansion_file_path, 'x')
        target_file.write(f"sequence,label\n")
        df = pd.read_csv(gene_file_path)
        print(f"Working on {c} {gene_file_path}                                 ", end='\r')
        for _, row in df.iterrows():
            seq_chunks = kmer(row['sequence'].strip(), _length, _stride)
            label_chunks = kmer(row['label'].strip(), _length, _stride)
            for seq, label in zip(seq_chunks, label_chunks):
                target_file.write(f"{seq},{label}\n")
        target_file.close()


In [ ]:
"""
Filter index based on gene name.
"""
import os
import pandas as pd
_chr_indices = [
    'NC_000001.11.csv',
    'NC_000002.12.csv',
    'NC_000003.12.csv',
    'NC_000004.12.csv',
    'NC_000005.10.csv',
    'NC_000006.12.csv',
    'NC_000007.14.csv',
    'NC_000008.11.csv',
    'NC_000009.12.csv',
    'NC_000010.11.csv',
    'NC_000011.10.csv',
    'NC_000012.12.csv',
    'NC_000013.11.csv',
    'NC_000014.9.csv',
    'NC_000015.10.csv',
    'NC_000016.10.csv',
    'NC_000017.11.csv',
    'NC_000018.10.csv',
    'NC_000019.10.csv',
    'NC_000020.11.csv',
    'NC_000021.9.csv',
    'NC_000022.11.csv',
    'NC_000023.11.csv',
    'NC_000024.10.csv']
_chr_dir = ["chr{}".format(i+1) for i in range(len(_chr_indices))]

for chr, chr_number in zip(_chr_indices, _chr_dir):
    path = os.path.join('data', 'genome', 'grch38', 'csvs_strand', chr)
    df = pd.read_csv(path)
    genes = list(df['gene'].unique())
    genes = [a for a in genes if not pd.isnull(a)]
    for g in genes:
        ndf = df[df['gene'] == g]
        path = os.path.join('data', 'genome', 'grch38', 'genes', chr_number,"{}.csv".format(g))
        if not os.path.exists(os.path.dirname(path)):
            os.mkdir(os.path.dirname(path))
        ndf.to_csv(path, index=False)
        print("Success: {}                                                      ".format(path), end="\r")

In [8]:
"""
Generate sequence from fasta based on gene indices.
"""
import os
from Bio import SeqIO
import pandas as pd
from tqdm import tqdm
from data_dir import chr_fasta_mapname

strand = '+'
chrs = ['chr{}'.format(i+1) for i in range(24)] # Folder name.
# chrs = ['chr{}'.format(i+1) for i in range(2)] # Folder name.

for c in chrs:
    chr_genes_indices_dir = os.path.join("data", "genome", "grch38", "genes", c)
    chr_genes_sequence_csv_dir = os.path.join("data", "genome", "seqlab.positive.strand",c)
    chr_fasta = os.path.join("data", "chr", chr_fasta_mapname[c])

    if not os.path.exists(chr_fasta):
        raise FileNotFoundError("Fasta {} not found.".format(chr_fasta))

    """
    Read whole chromosome here and return its complete sequence.
    Yes, it's long.
    """
    records = SeqIO.parse(chr_fasta, "fasta")
    chr_records = next(records)
    chr_sequence = str(chr_records.seq)
    genome_sequence = chr_sequence # Use genome sequence reader here.

    _columns = ['sequence', 'label']
    for fname in os.listdir(chr_genes_indices_dir):
        print("Working on chr {} gene {}                                    ".format(c, fname), end='\r')
        fpath = os.path.join(chr_genes_indices_dir, fname)
        if os.path.isfile(fpath):
            """
            Gene index found. Read the index and cross-reference with genome sequence.
            """
            index_df = pd.read_csv(fpath)
            gene_region = index_df[index_df['region'] == "gene"]
            if strand != None:
                gene_region = gene_region[gene_region['strand'] == strand]

            if len(gene_region) > 0:
                gene_df = pd.DataFrame(columns=_columns)
                for i, g in gene_region.iterrows():
                    gene_start_index = int(g['start_index'])
                    gene_end_index = int(g['end_index'])
                    gene_sequence = genome_sequence[gene_start_index:gene_end_index + 1]
                    # print("Gene sequence: {}".format(gene_sequence))
                    # If gene sequence isn't None then this gene is available in chromosome sequence.
                    if gene_sequence != None:
                        gene_sequential_labelling = ['i' for a in gene_sequence]
                        exons = index_df[index_df['region'] == "exon"]
                        #if strand != None:
                        #    exons = index_df[index_df['strand'] == strand]
                        for j, r in exons.iterrows():
                            start_index = int(r['start_index'])
                            end_index = int(r['end_index'])
                            if (start_index >= gene_start_index and end_index <= gene_end_index):
                                start_index = int(r['start_index']) - gene_start_index
                                end_index = int(r['end_index']) - gene_start_index
                                for k in range(start_index, end_index + 1):
                                    gene_sequential_labelling[k] = 'E'
                        gene_sequential_labelling = ''.join(gene_sequential_labelling)
                        #endfor
                        gene_df = pd.concat([gene_df, pd.DataFrame([[gene_sequence, gene_sequential_labelling]], columns=_columns)])

                target_path = os.path.join(chr_genes_sequence_csv_dir, fname)
                # data\genome\sequential-labelling\chr1
                # Only write if dataframe is not empty.
                if gene_df.shape[0] > 0:
                    if not os.path.exists(chr_genes_sequence_csv_dir):
                        os.makedirs(chr_genes_sequence_csv_dir, exist_ok=True)
                    if gene_df.shape[0] > 0:    
                        gene_df.to_csv(target_path, index=False)
                    

In [1]:
"""Chunk each gene into 512 characters, for each chromosome."""
import pandas as pd
from data_preparation import kmer
import os

chrs = [f"chr{i + 1}" for i in range(24)] # Test one chromosome.
chr_paths = [os.path.join("data", "genome", "seqlab.positive.strand", f"{chr}") for chr in chrs]
dest_paths = [os.path.join("data", "genome", "seqlab.strand-positive.stride-512", f"{chr}") for chr in chrs]
for cp, dp in zip(chr_paths, dest_paths):
    genes = os.listdir(cp)
    genes = [g for g in genes if "expanded" not in g.split('.')]
    for gene in genes:
        gene_path = os.path.join(cp, gene)
        gene_df = pd.read_csv(gene_path)
        print(f"Processing {os.path.basename(cp)} {gene}                    ", end="\r")
        for i, r in gene_df.iterrows():
            sequence = r["sequence"]
            label = r["label"]
            seq_chunks = kmer(sequence, 512, 512)
            label_chunks = kmer(label, 512, 512)
            dest_path = os.path.join(dp, f"{gene.split('.')[0]}.csv")
            if os.path.exists(dest_path):
                os.remove(dest_path)
            if not os.path.exists(os.path.dirname(dest_path)):
                os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            dest = open(dest_path, "x")
            dest.write("sequence,label\n")
            for c, l in zip(seq_chunks, label_chunks):
                dest.write(f"{c},{l}\n")
            dest.close()


In [ ]:
"""Move file around"""

import os
for i in range(24):
    k = i + 1
    src_path = os.path.join("data", "genome", "seqlab.positive.strand", f"chr{k}")
    dest_path = os.path.join("data", "genome", "seqlab.positive.strand", f"chr{k}.expanded")
    if not os.path.exists(dest_path):
        os.makedirs(dest_path, exist_ok=True)
    files = os.listdir(src_path)
    files = [f for f in files if "expanded" in f.split('.')]
    for f in files:
        oldpath = os.path.join(src_path, f)
        newpath = os.path.join(dest_path, f)
        if not os.path.exists(newpath):
            os.rename(oldpath, newpath)
        else:
            print(f"Skip {oldpath}                                                    ", end="\r")

In [ ]:
"""
Rename files in expanded folder
filename.expanded.csv => filename.csv
"""
src_paths = [os.path.join("data", "genome", "seqlab.positive.strand", f"chr{i + 1}.expanded") for i in range(24)]
for srcdir in src_paths:
    files = os.listdir(srcdir)
    for f in files:
        oldpath = os.path.join(srcdir, f)
        newpath = os.path.join(srcdir, f"{f.split('.')[0]}.csv")
        os.rename(oldpath, newpath)

In [4]:
"""Generate kmer version from sequence."""
import os
import pandas as pd
from data_preparation import str_kmer

# src_paths = [os.path.join("data", "genome", "seqlab.positive.strand", f"chr{i + 1}") for i in range(24)]
src_paths = [os.path.join("data", "genome", "seqlab.strand-positive.stride-512", f"chr{i + 1}") for i in range(24)]
# dest_paths = [os.path.join("data", "genome", "seqlab.strand-positive.kmer", f"chr{i + 1}") for i in range(24)]
dest_paths = [os.path.join("data", "genome", "seqlab.strand-positive.kmer.stride-512", f"chr{i + 1}") for i in range(24)]
for srcdir, destdir in zip(src_paths, dest_paths):
    files = os.listdir(srcdir)
    for f in files:
        src = os.path.join(srcdir, f)
        dest = os.path.join(destdir, f)
        if not os.path.exists(destdir):
            os.makedirs(destdir, exist_ok=True)
        if os.path.exists(dest):
            os.remove(dest)
        dest = open(dest, "x")
        dest.write("sequence,label\n")
        df = pd.read_csv(src)
        for i, r in df.iterrows():
            sequence = r["sequence"]
            label = r["label"]
            dest.write(f"{str_kmer(sequence, 3)},{str_kmer(label, 3)}\n")
        dest.close()
            

In [1]:
"""Chunk each gene into 510 token. Since sequence is already in token form, ``kmer`` function cannot be used, had to create another script."""
import pandas as pd
from data_preparation import kmer
import os

def chunk_kmer_sequence(chunk: str, size: int, stride: int) -> str:
    arr = chunk.split(' ')
    arr = kmer(arr, size, window_size=stride)
    return arr

chrs = [f"chr{i + 1}" for i in range(24)] # Test one chromosome.
chr_paths = [os.path.join("data", "genome", "seqlab.strand-positive.kmer", f"{chr}") for chr in chrs]
dest_paths = [os.path.join("data", "genome", "seqlab.strand-positive.kmer.stride-205", f"{chr}") for chr in chrs]
for cp, dp in zip(chr_paths, dest_paths):
    genes = os.listdir(cp)
    genes = [g for g in genes if "expanded" not in g.split('.')]
    for gene in genes:
        gene_path = os.path.join(cp, gene)
        gene_df = pd.read_csv(gene_path)
        print(f"Processing {os.path.basename(cp)} {gene}                    ", end="\r")
        for i, r in gene_df.iterrows():
            sequence = r["sequence"]
            label = r["label"]
            seq_chunks = chunk_kmer_sequence(sequence, 510, 205)
            label_chunks = chunk_kmer_sequence(label, 510, 205)    
            dest_path = os.path.join(dp, f"{gene.split('.')[0]}.csv")
            if os.path.exists(dest_path):
                os.remove(dest_path)
            if not os.path.exists(os.path.dirname(dest_path)):
                os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            dest = open(dest_path, "x")
            dest.write("sequence,label\n")
            for c, l in zip(seq_chunks, label_chunks):
                dest.write(f"{' '.join(c)},{' '.join(l)}\n")
            dest.close()

In [2]:
"""
Generate gene index in CSV ('chr', 'gene') to see if dataset split can be done by genes.
"""
import pandas as pd
import os

strand = os.path.join("data", "genome", "seqlab.strand-positive.kmer.stride-205")
chrs = [os.path.join(strand, f"chr{i+1}") for i in range(24)]
_cols = ["chr", "gene"]
df = pd.DataFrame(columns=_cols)
for chrdir in chrs:
    files = os.listdir(chrdir)
    for f in files:
        df = pd.concat([df, 
            pd.DataFrame([[f"{os.path.basename(chrdir)}", f]], columns=_cols)
        ])
gene_index = os.path.join(strand, f"gene_index.csv")
if os.path.exists(gene_index):
    os.remove(gene_index)
df.to_csv(gene_index, index=False)

In [4]:
"""
Split gene_index for training, validation, and testing.
"""
import os
strand = os.path.join("data", "genome", "seqlab.strand-positive.kmer.stride-205")
gene_index = os.path.join(strand, f"gene_index.csv")
train_index = os.path.join(strand, "gene_train_index.csv")
val_index = os.path.join(strand, "gene_validation_index.csv")
test_index = os.path.join(strand, "gene_test_index.csv")

import pandas as pd
df = pd.read_csv(gene_index)
train_df = df.sample(frac=0.8)
test_df = df.drop(train_df.index)
val_df = train_df.sample(frac=0.2)
train_df = train_df.drop(val_df.index)
train_df.to_csv(train_index, index=False)
val_df.to_csv(val_index, index=False)
test_df.to_csv(test_index, index=False)

In [ ]:
import pandas as pd
_cols = ['chr', 'gene']
df_path = os.path.join('workspace', 'sequential-labelling', 'gene_index.csv')
_dir = os.path.join('workspace', 'sequential-labelling', 'duplicate genes')
os.makedirs(_dir, exist_ok=True)
df = pd.read_csv(df_path)
train_df = pd.DataFrame(columns=_cols)
valid_df = pd.DataFrame(columns=_cols)
test_df = pd.DataFrame(columns=_cols)
genes_unique = df['gene'].unique()
for g in genes_unique:
    filtered_df = df[df['gene'] == g]
    if filtered_df.shape[0] > 1:
        _g = "{}.csv".format(g)
        filtered_df.to_csv(os.path.join(_dir, _g), index=False)

In [ ]:
from data_preparation import gff_to_csvs, gff_to_csv
from data_dir import (annotated_grch38_gff, annotated_grch38_gff_dir, annotated_grch38_gff_csv)

print(annotated_grch38_gff)
print(annotated_grch38_gff_csv)
print(annotated_grch38_gff_dir)

In [ ]:
from data_dir import chr21_fasta, chr21_index_csv, data_genome_grch38_labels_dir
from data_preparation import generate_sequence_labelling
import os

target_path = os.path.join(data_genome_grch38_labels_dir, 'chr21.csv')
print("Generate sequential labelling {} => {}: {}".format(chr21_index_csv, target_path, generate_sequence_labelling(chr21_index_csv, chr21_fasta, target_path)))


In [ ]:
from data_dir import (
    chr1_index_csv, chr2_index_csv, chr3_index_csv, chr4_index_csv, chr5_index_csv, chr6_index_csv, chr7_index_csv, chr8_index_csv, chr9_index_csv, chr10_index_csv,
    chr11_index_csv, chr12_index_csv, chr13_index_csv, chr14_index_csv, chr15_index_csv, chr16_index_csv, chr17_index_csv, chr18_index_csv, chr19_index_csv, chr20_index_csv,
    chr21_index_csv, chr22_index_csv, chr23_index_csv, chr24_index_csv
)
from data_dir import (
    chr1_fasta, chr2_fasta, chr3_fasta, chr4_fasta, chr5_fasta, chr6_fasta, chr7_fasta, chr8_fasta, chr9_fasta, chr10_fasta, 
	chr11_fasta, chr12_fasta, chr13_fasta, chr14_fasta, chr15_fasta, chr16_fasta, chr17_fasta, chr18_fasta, chr19_fasta, chr20_fasta, 
	chr21_fasta, chr22_fasta, chr23_fasta, chr24_fasta,
)
chr_fastas = [
    chr1_fasta,
	#chr2_fasta, chr3_fasta, chr4_fasta, chr5_fasta, chr6_fasta, chr7_fasta, chr8_fasta, chr9_fasta, chr10_fasta,
	#chr11_fasta, chr12_fasta, chr13_fasta, chr14_fasta, chr15_fasta, chr16_fasta, chr17_fasta, chr18_fasta, chr19_fasta, chr20_fasta,
	#chr21_fasta, chr22_fasta, chr23_fasta, chr24_fasta
]
from data_dir import labseq_dir, labseq_names
from data_preparation import generate_sequence_labelling
chr_indices = [
    chr1_index_csv, 
	#chr2_index_csv, chr3_index_csv, chr4_index_csv, chr5_index_csv, chr6_index_csv, chr7_index_csv, chr8_index_csv, chr9_index_csv, chr10_index_csv,
    #chr11_index_csv, chr12_index_csv, chr13_index_csv, chr14_index_csv, chr15_index_csv, chr16_index_csv, chr17_index_csv, chr18_index_csv, chr19_index_csv, chr20_index_csv,
    #chr21_index_csv, chr22_index_csv, chr23_index_csv, chr24_index_csv
]
chr_labseq_path = [os.path.join(labseq_dir, fname) for fname in labseq_names[0:1]]
for src, fasta, target in zip(chr_indices, chr_fastas, chr_labseq_path):
    print(src, fasta, target)
    #print("Generating sequential labelling for index {}, from fasta {}, to {}: {}".format(src, fasta, target, generate_sequence_labelling(src, fasta, target, do_expand=True, expand_size=512)))

In [ ]:
from sequential_labelling import Label_Dictionary
import pandas as pd
from tqdm import tqdm
import os

def prepare_sequence_from_csv(src_csv, label_dictionary=Label_Dictionary):
    """
    Convert sequence into tokenized DNA sequence and label sequence.
    CSV source has columns `sequence` and `label`. 
    `sequence` contains tokenized DNA sequence and `label` contains sequence of labels.
    @param      src_csv (string): path to CSV source.
    @param      label_dictionary (dict): dictionary to convert label into number.
    @return     sequence, labels
    """
    if not os.path.exists(src_csv):
        raise FileNotFoundError(src_csv)
    if label_dictionary == None:
        raise Exception("Argument `label_dictionary` cannot be empty!")
    
    df = pd.read_csv(src_csv)
    return list(df['sequence']), list(df['label'])
    

In [ ]:
"""Create bundle from all expanded sequence from all genes from all chromosomes."""
import os
import pandas as pd
from tqdm import tqdm

src_dirs = [os.path.join("data", "genome", "seqlab.positive.strand", f"chr{i+1}.expanded") for i in range(24)]
bundle_path = os.path.join("data", "genome", "seqlab.positive.strand", "bundle.csv")
if os.path.exists(bundle_path):
    os.remove(bundle_path)
bundle = open(bundle_path, "x")
bundle.write("sequence,label\n")
for srcdir in tqdm(src_dirs, total=24):
    files = [os.path.join(srcdir, f) for f in os.listdir(srcdir)]
    for f in files:
        df = pd.read_csv(f)
        for i, r in df.iterrows():
            bundle.write(f"{r['sequence']},{r['label']}\n")
bundle.close()

In [1]:
# 17 May 2022
# From non overlapping kmer-ized genes, make bundle.
import pandas as pd
import os
chrs = [f"chr{i + 1}" for i in range(24)]
srcs = [os.path.join("workspace", "seqlab", "by_genes", "gene_dir", "seqlab.strand-positive.kmer.stride-510", chr) for chr in chrs]
dest = os.path.join("workspace", "seqlab", "by_sequence", "seqlab.strand-positive.kmer.stride-510", "bundle.csv")

if os.path.exists(dest):
    os.remove(dest)
fdest = open(dest, "x")
fdest.write("sequence,label\n")
for src in srcs:
    files = os.listdir(src)
    files = [os.path.join(src, f) for f in files]
    for f in files:
        df = pd.read_csv(f)
        for i,r in df.iterrows():
            fdest.write(f"{r['sequence']},{r['label']}\n")


fdest.close()

In [3]:
# Creating bundle from index.
import os
import pandas as pd
path = os.path.join("workspace", "seqlab", "by_genes", "gene_dir", "seqlab.strand-positive.kmer.stride-510")
train_index = os.path.join(path, "gene_train_index.csv")
validation_index = os.path.join(path, "gene_validation_index.csv")
test_index = os.path.join(path, "gene_test_index.csv")

# Create training bundle from index.
# Store file in by_sequence folder.
bundle_path = os.path.join("workspace", "seqlab", "by_sequence", "seqlab.strand-positive.kmer.stride-510.from-index")
train_bundle = os.path.join(bundle_path, "gene_train_bundle.csv")
validation_bundle = os.path.join(bundle_path, "gene_validation_bundle.csv")
test_bundle = os.path.join(bundle_path, "gene_test_bundle.csv")
for s, d in zip([train_index, validation_index, test_index], [train_bundle, validation_bundle, test_bundle]):
    df = pd.read_csv(s)
    if os.path.exists(d):
        os.remove(d)
    dest_file = open(d, "x")
    dest_file.write("sequence,label\n")
    for i, r in df.iterrows():
        chr = r["chr"]
        gene = r["gene"]
        gene_path = os.path.join(path, chr, gene)
        gene_df = pd.read_csv(gene_path)
        for j, k in gene_df.iterrows():
            dest_file.write(f"{k['sequence']},{k['label']}\n")

    dest_file.close()


In [2]:
"""Split the bundle into training, validation, and testing bundle."""
import pandas as pd
import os
# bundle_path = os.path.join("data", "genome", "seqlab.positive.strand", "bundle.csv")
dirname = os.path.join("workspace", "seqlab", "by_sequence", "seqlab.strand-positive.kmer.stride-510")
bundle_path = os.path.join(dirname, "bundle.csv")
train_bundle_path = os.path.join(dirname, "train_bundle.csv")
val_bundle_path = os.path.join(dirname, "val_bundle.csv")
test_bundle_path = os.path.join(dirname, "test_bundle.csv")
bundle_df = pd.read_csv(bundle_path)
train_bundle_df = bundle_df.sample(frac=0.8)
test_bundle_df = bundle_df.drop(train_bundle_df.index)
val_bundle_df = train_bundle_df.sample(frac=0.2)
train_bundle_df = train_bundle_df.drop(val_bundle_df.index)
train_bundle_df.to_csv(train_bundle_path, index=False)
val_bundle_df.to_csv(val_bundle_path, index=False)
test_bundle_df.to_csv(test_bundle_path, index=False)

In [7]:
import torch

pred = torch.rand(4, 512, 11)
label = torch.randint(0, 11, (4, 512, ))

print(pred)
print(label)

loss = torch.nn.CrossEntropyLoss()(pred, label)
print(loss)

tensor([[[0.2956, 0.7296, 0.8612,  ..., 0.4464, 0.4151, 0.1572],
         [0.2750, 0.2984, 0.9251,  ..., 0.3523, 0.5679, 0.6113],
         [0.4516, 0.7322, 0.0482,  ..., 0.8788, 0.3193, 0.5869],
         ...,
         [0.8682, 0.6648, 0.3179,  ..., 0.6589, 0.0543, 0.5085],
         [0.5889, 0.2499, 0.2031,  ..., 0.2418, 0.4316, 0.3308],
         [0.1615, 0.3963, 0.1764,  ..., 0.6691, 0.1192, 0.3109]],

        [[0.9314, 0.4004, 0.9344,  ..., 0.7260, 0.2379, 0.3811],
         [0.2650, 0.2672, 0.6284,  ..., 0.0849, 0.0414, 0.3982],
         [0.5978, 0.8488, 0.7534,  ..., 0.1637, 0.7735, 0.6409],
         ...,
         [0.9666, 0.0571, 0.9573,  ..., 0.1123, 0.9359, 0.2091],
         [0.1656, 0.0606, 0.6611,  ..., 0.7447, 0.5924, 0.1701],
         [0.8858, 0.7724, 0.3839,  ..., 0.0140, 0.9449, 0.1008]],

        [[0.3420, 0.4666, 0.9131,  ..., 0.0446, 0.8449, 0.5538],
         [0.6441, 0.3585, 0.0303,  ..., 0.1870, 0.9063, 0.9853],
         [0.7677, 0.3281, 0.0045,  ..., 0.4886, 0.6021, 0.

RuntimeError: Expected target size [4, 11], got [4, 512]